## Community Detection-based Node Importance for Identifying Coding Genes

This notebook contains code for identifying important nodes in the cancer network. The cancer network was built in [1]. In this work, a community detection algorithm is used to partition the cancer network into communities and a centrality-based metric is implemented to compute the importance of each node in the network [2]. Specifically, the well-known Louvain algorithm is used for detecting communities, and a centrality-based metric is used to compute the importance of each node with respect to its community. 

The following steps are performed to compute the node importance and validate the important nodes,
1. Build an undirected graph G using the edge list of the cancer network.
2. Partition the graph into communities using the Louvain algorithm.
3. Compute eigenvectors of the adjancency matrix of graph G.
4. Compute node importance for each node in G using a centrality-based metric.
5. Sort the coding genes in the cancer network in descending order.
6. Validate the coding genes with gold standard CGC.

### References:
1. [Pham, Vu VH, Lin Liu, Cameron P. Bracken, Gregory J. Goodall, Qi Long, Jiuyong Li, and Thuc D. Le. "CBNA: A control theory based method for identifying coding and non-coding cancer drivers." PLoS Computational Biology 15, no. 12 (2019).](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007538#sec009)
2. [Wang, Y., Di, Z., & Fan, Y. (2011). Identifying and characterizing nodes important to community structure using the spectrum of the graph. PloS one, 6(11).](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0027418)

In [1]:
%load_ext autoreload
%autoreload 2

# Standard libraries
from os.path import join

# Libraries for graphs
import community
import networkx as nx

# Libraries for matrix computations
import numpy as np
import pandas as pd

# Libraries for sparse matrices and eigenvectors
from scipy.sparse import diags
from scipy.sparse.linalg import eigs
from scipy.stats import pearsonr

In [32]:
import logging

In [ ]:
logging.ge

In [2]:
def compute_node_importance(n_nodes, n_communities, eig_vectors):
    """
    Computes node importance using a centrality-based metric which uses the eigenvectors of the adjacency matrix.
    Args:
        n_nodes: Number of nodes in the graph.
        n_communities: Number of communities estimated.
        eig_vectors: np.ndarray of size n x c where n is number of nodes and c is the number of communities.
        The eigenvectors are computed from the adjacency matrix and are stored as columns.
    Returns: 
        np.array: A numpy array containing the node importance score normalized by the
        number of communities.
    """
    p_k = [0] * n_nodes
    I = [0] * n_nodes
    for k in range(n_nodes):
        for i in range(n_communities):
            p_k[k] += eig_vectors[k, i]**2/np.sum(eig_vectors[:, i]**2)
        I[k] = p_k[k]/n_communities
    return np.array(I)

In [3]:
DATA_DIR = "/home/mandar/Data/NCSU/CBNA/cbna-community-detection/Data/"
OUT_DIR = "/home/mandar/Data/NCSU/CBNA/cbna-community-detection/Output/"

In [4]:
# Read cancer network data
cancer_network = pd.read_csv(join(DATA_DIR, "pVal_cancer_network.csv"))

# Read cancer data
cancer_data = pd.read_csv(join(DATA_DIR, "cancer_data.csv"))

# Load gold standard CGC
gold_standard_cgc_df = pd.read_csv(join(DATA_DIR, "Census_allFri Sep 28 07_39_37 2018.tsv"), sep="\t")
gold_standard_cgc = gold_standard_cgc_df["Gene Symbol"]

# Load mRNAs column names from BRCA_matchedData RData file
mRNAs_df = pd.read_csv(join(DATA_DIR, "mRNAs_data.csv"))

In [5]:
cancer_data.head()

,Unnamed: 0,hsa-let-7a-5p,hsa-let-7b-5p,hsa-let-7c-5p,hsa-let-7d-5p,hsa-let-7e-5p,hsa-let-7f-5p,hsa-let-7g-5p,hsa-let-7i-5p,hsa-miR-1-3p,...,ZNF764,ZNF768,ZNF775,ZNF8,ZNF83,ZNF91,ZSCAN16,ZSCAN20,ZSCAN21,ZXDC
0,TCGA-EW-A1P4,12.994388,14.378198,11.546615,10.651545,10.807892,8.403284,9.420617,10.154370,1.191638,...,2.533052,5.790201,1.053708,2.901450,4.655640,3.708116,2.209807,2.001186,2.641918,5.656499
1,TCGA-OL-A5D7,12.222621,12.889247,9.436776,10.246441,8.817331,9.100132,10.788659,9.041260,0.247525,...,3.697387,6.229218,3.352287,2.466736,3.581223,2.964224,2.853394,1.076350,3.887751,5.704944
2,TCGA-GM-A2DI,14.788013,15.883161,12.544769,8.582312,11.219828,9.657708,10.197309,9.485176,0.871444,...,4.306762,6.314290,1.703591,2.814682,4.877576,5.657588,2.743309,2.883605,2.695927,5.954116
3,TCGA-D8-A1XC,13.421196,15.264008,7.830062,8.476702,9.545340,7.231684,8.672442,7.872247,-0.391719,...,4.557417,6.289612,2.691002,2.562511,5.205412,5.301122,2.332271,3.585772,3.227813,6.207622
4,TCGA-E2-A1AZ,12.602671,13.497989,12.312566,8.514709,9.765894,7.788895,7.858109,9.086874,0.195002,...,2.628138,5.856284,-0.111058,3.106091,5.975766,6.336993,3.025550,2.481297,4.025336,6.262658


### Compute correlation between every pair of edges

In [6]:
def computePearsonCorrelation(cancer_network: pd.DataFrame, cancer_data: pd.DataFrame) -> pd.DataFrame:
    """
    This function computes pearson correlation coefficient for every pair of adjacent nodes in the cancer
    network. The correlation coefficient is stored in the `correlation` column and its absolute value is
    stored in the `weight` column. 
    Args:
        cancer_network: Pandas dataframe containing edge list of the cancer network.
        cancer_data: Pandas dataframe containing the expression level of nodes in the cancer network.
    Returns:
        pd.DataFrame containing 3 more columns added to cancer_network dataframe. 
    """
    correlation_coeffs = []
    correlation_pvals = []
    x = cancer_network.cause.values.tolist()
    y = cancer_network.effect.values.tolist()

    for x_i, y_i in zip(x, y):
        corr, pval = pearsonr(cancer_data.loc[:, x_i], cancer_data.loc[:, y_i])
        correlation_coeffs.append(corr)
        correlation_pvals.append(pval)
    cancer_network["correlation"] = pd.Series(correlation_coeffs)
    cancer_network["pval"] = pd.Series(correlation_pvals)
    cancer_network["weight"] = cancer_network["correlation"].abs()
    return cancer_network

In [7]:
cancer_network = computePearsonCorrelation(cancer_network, cancer_data)
cancer_network.head()

,cause,effect,correlation,pval,weight
0,A1BG,ANXA7,-0.195811,6.848169e-08,0.195811
1,A1CF,SYNCRIP,0.096508,8.304506e-03,0.096508
2,A2M,ANXA6,0.167517,4.157256e-06,0.167517
3,A2M,ANXA7,-0.111640,2.245310e-03,0.111640
4,A2M,ATF7IP,0.191320,1.371506e-07,0.191320


### Step 1: Build an undirected graph G using the edge list of the cancer network

In [8]:
def buildGraphFromEdgeList(cancer_network: pd.DataFrame, is_weighted=False) -> nx.Graph:
    """
    This function builds a networkx graph object from the edge list. A boolean option is provided to build
    a weighted undirected graph.
    Args:
        cancer_network: Pandas dataframe containing edge list of the cancer network.
        is_weighted: boolean specifying whether a weighted graph should be created.
    Returns:
        nx.Graph object containing undirected edges between pairs of nodes specified in cancer_network dataframe.
    """
    if is_weighted:
        try:
            # Build graph by reading the edge list of the cancer network
            G = nx.from_pandas_edgelist(cancer_network, source="cause", target="effect", edge_attr=['weight'])
        except:
            raise ValueError("Dataframe does not contain a column containing edge weights")
    else:
        # Build graph by reading the edge list of the cancer network
        G = nx.from_pandas_edgelist(cancer_network, source="cause", target="effect")
    print("Number of nodes: {0}, and number of edges: {1}".format(len(G.nodes), len(G.edges)))
    print("Graph weights: {0}".format(G.size(weight='weight')))
    return G

In [19]:
G = buildGraphFromEdgeList(cancer_network, is_weighted=True)

Number of nodes: 7357, and number of edges: 125867
Graph weights: 23940.100269972296


### Step 2: Partition the graph into communities using the Louvain algorithm

In [20]:
def performCommunityDetection(G: nx.Graph) -> int:
    """
    This function performs community detection on graph G using the Louvain algorithm.
    Args:
        G: nx.Graph object representing an undirected graph
    Returns:
        int specifying the number of communities detected in the graph.
    """
    # Run community detection algorithm on graph G
    partition = community.best_partition(G)
    n_communities = len(set(partition.values()))
    
    print(f"Number of communities in G: {n_communities}")
    
    return n_communities

In [21]:
n_communities = performCommunityDetection(G)

Number of communities in G: 13


### Step 3: Compute eigenvectors of the adjancency matrix of graph G

In [22]:
def computeEigenvectors(G: nx.Graph, n_communities: int) -> np.ndarray:
    """
    This function computes the eigenvectors of the adjacency matrix. The number of eigenvectors returned 
    is specified by the n_communities integer. 
    """
    # Create adjacency matrix of graph G
    A = nx.adjacency_matrix(G)
    
    # Compute eigenvalues and eigenvectors of the adjacency matrix A
    _, adj_eig_vectors = eigs(A.toarray().astype(float), k=n_communities, which='LR')
    
    return adj_eig_vectors

In [23]:
adj_eigen_vectors = computeEigenvectors(G, n_communities)

### Step 4: Compute node importance for each node in G using a centrality-based metric

In [24]:
# Compute node importance index I using the eigenvectors of the adjacency matrix
I = compute_node_importance(G.number_of_nodes(), n_communities, np.real(adj_eigen_vectors))

## Select top-K important nodes in the network

### Step 5: Sort the coding genes in the cancer network in descending order

In [25]:
def buildNodeImportanceDataFrame(G: nx.Graph, I: np.array) -> pd.DataFrame:
    """
    This function builds a dataframe containing two columns. First column contains the names of the nodes in
    graph G and second column contains their importance score. The nodes are sorted in descending order
    of their importance score.
    Args:
        G: networkx graph object
        I: numpy array containing importance score.
    Returns:
        pd.DataFrame containing node names and their importance score.
    """
    # Get list of all nodes in the cancer network
    nodes = list(G.nodes)
    # Sort the nodes by their importance score in descending order
    important_nodes = [nodes[i] for i in np.argsort(I)[::-1]]
    node_importance = np.sort(I)[::-1]
    # Build a dataframe with two columns, one containing node name and second containing importance score
    node_importance_df = pd.concat([pd.Series(important_nodes), pd.Series(node_importance)], axis=1)
    node_importance_df.columns = ['node', 'importance']
    return node_importance_df

In [26]:
node_importance_df = buildNodeImportanceDataFrame(G, I)

### Step 6: Validate the coding genes with gold standard CGC

In [27]:
def validateTopKCodingGenes(node_importance_df: pd.DataFrame, mRNAs_df: pd.DataFrame, gold_standard_cgc: pd.Series):
    """
    This function performs the following steps,
        1. Find the coding genes in the cancer network by matching the node names with those in mRNAs data
        2. Select the top-K coding genes found in the cancer network and validate them against the gold standard cgc.
        3. Save the results to a csv file.
    Args:
        node_importance_df: Pandas dataframe containig nodes sorted in descending order of their importance
        scores.
        mRNAs_data: Pandas dataframe with columns containing names of coding genes.
        gold_standard_cgc: Pandas series containing gold standard coding genes
    """
    # Step 1: Find coding genes in the cancer network
    # Perform intersection of the nodes in the cancer network and the mRNAs
    coding_genes = node_importance_df.loc[node_importance_df.node.isin(mRNAs_df.columns), ]
    # Select top-K coding genes and save the results to csv files
    top_k = [50, 100, 150, 200]
    for K in top_k:
        top_k_coding_genes = coding_genes.iloc[:K, ].copy()
        # Step 2: Validate top-k coding genes against the gold standard cgc
        coding_genes_gold_standard = top_k_coding_genes.loc[top_k_coding_genes.node.isin(gold_standard_cgc)]
        print("Coding genes in top-{0}: {1}".format(K, coding_genes_gold_standard.shape[0]))
        
        # Step 3: Save the results in a csv file
        coding_genes_gold_standard.to_csv(join(OUT_DIR, "top_{0}_coding_genes_gold_standard_validation.csv".format(K)))

In [28]:
validateTopKCodingGenes(node_importance_df, mRNAs_df, gold_standard_cgc)

Coding genes in top-50: 24
Coding genes in top-100: 47
Coding genes in top-150: 65
Coding genes in top-200: 76
